<a href="https://colab.research.google.com/github/Abhisheksingh12345/ADT/blob/master/dev1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=de242cfdb73bd372c3a984f159bf2f505d122a417d550d45a7f0b6e1fae830f8
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import matplotlib.pyplot as plts

In [6]:
spark = SparkSession.builder.appName('dev_1').master('local[4]').getOrCreate()

In [7]:
sc = spark.sparkContext

In [8]:
table_A_path = '/content/drive/MyDrive/datasets/Table_A.csv'
table_B_path = '/content/drive/MyDrive/datasets/Table_B.csv'
table_C_path = '/content/drive/MyDrive/datasets/Table_C.csv'

In [9]:
table_A_df = spark.read.format('csv').option('inferSchema',True).option('header',True).load(table_A_path)

In [10]:
table_B_df = spark.read.format('csv').option('inferSchema',True).option('header',True).load(table_B_path)

In [11]:
table_C_df = spark.read.format('csv').option('inferSchema',True).option('header',True).load(table_C_path)

In [12]:
table_A_df.show()

+------+----------+------+----------+------------+
|cif_id|account_no|amount|department|Pride_rating|
+------+----------+------+----------+------------+
|  1222|       A01| 50000|       C2E|         3.0|
|  1353|       A03| 50000|       MOT|         1.0|
|  1262|       B01|100000|       IIT|         2.5|
|  1373|       BC2|300000|       C2E|         1.5|
+------+----------+------+----------+------------+



In [13]:
table_B_df.show()

+------+-------+-----------+---------+---------------------+
|cif_id|loan_no|loan_amount|loan_type|last_date_for_payment|
+------+-------+-----------+---------+---------------------+
|  1222|    A01|     100000|     HOME|           2023-09-31|
|  1353|    A03|     200000| PERSONAL|           2024-01-02|
|  1046|    C21|     100000|      CAR|           2023-10-11|
|  1400|    C23|     300000|     HOME|           2023-10-15|
|  1501|    D12|     400000|EDUCATION|           2024-01-02|
+------+-------+-----------+---------+---------------------+



In [14]:
table_C_df.show()

+-------+----------+------------+-----------+
|acct_no|saving_acc|current_acct|  open_date|
+-------+----------+------------+-----------+
|     12|     20000|       50000| 01-07-2023|
|     21|     30000|       60000| 23-06-2022|
|     31|     30000|        null| 2021-05-23|
|     13|    100000|        null| 02-07-2023|
|     41|      null|        null| 10-01-2023|
|     14|    320000|      600000| 05-01-2022|
+-------+----------+------------+-----------+



In [15]:
withoutscpace_tableC_df = table_C_df.na.fill("").withColumn("open_date",date_format(to_date(col("open_date"), "dd-MM-yyyy"), "yyyy-MM-dd"))

In [16]:
withoutscpace_tableC_df.show()

+-------+----------+------------+----------+
|acct_no|saving_acc|current_acct| open_date|
+-------+----------+------------+----------+
|     12|     20000|       50000|2023-07-01|
|     21|     30000|       60000|2022-06-23|
|     31|     30000|        null|      null|
|     13|    100000|        null|2023-07-02|
|     41|      null|        null|2023-01-10|
|     14|    320000|      600000|2022-01-05|
+-------+----------+------------+----------+



In [ ]:
#@title Q1

In [25]:
#@title Q2 Merge table A and table B if A or B, find second highest amount for each department
from pyspark.sql.window import Window
mergeA_B_df = table_A_df.join(table_B_df, on = 'cif_id', how = "full")
mergeA_B_2df = mergeA_B_df.coalesce(4)
window_spec = Window.partitionBy("department").orderBy(col("amount").desc())
merged_df = mergeA_B_2df.withColumn("row_num", row_number().over(window_spec))
secound_highest = merged_df.filter( col('department').isNull and (col('row_num') == 2))
secound_highest.show()

+------+----------+------+----------+------------+-------+-----------+---------+---------------------+-------+
|cif_id|account_no|amount|department|Pride_rating|loan_no|loan_amount|loan_type|last_date_for_payment|row_num|
+------+----------+------+----------+------------+-------+-----------+---------+---------------------+-------+
|  1400|      null|  null|      null|        null|    C23|     300000|     HOME|           2023-10-15|      2|
|  1222|       A01| 50000|       C2E|         3.0|    A01|     100000|     HOME|           2023-09-31|      2|
+------+----------+------+----------+------------+-------+-----------+---------+---------------------+-------+



In [35]:
#@title Q3 Merge table A table B if A, loan no is null and last_date payroll >= 2024 then delete whole row
Q_3_df = table_A_df.join(table_B_df, on=["cif_id"], how="left_outer") \
  .filter((table_B_df["loan_no"].isNull()) & (table_B_df["last_date_for_payment"] >= "2024-01-01"))
Q_3_df.show()
Q_3_= table_A_df.join(Q_3_df.select("cif_id"), on=["cif_id"], how="left_anti")
Q_3_.show()


+------+----------+------+----------+------------+-------+-----------+---------+---------------------+
|cif_id|account_no|amount|department|Pride_rating|loan_no|loan_amount|loan_type|last_date_for_payment|
+------+----------+------+----------+------------+-------+-----------+---------+---------------------+
+------+----------+------+----------+------------+-------+-----------+---------+---------------------+

+------+----------+------+----------+------------+
|cif_id|account_no|amount|department|Pride_rating|
+------+----------+------+----------+------------+
|  1222|       A01| 50000|       C2E|         3.0|
|  1353|       A03| 50000|       MOT|         1.0|
|  1262|       B01|100000|       IIT|         2.5|
|  1373|       BC2|300000|       C2E|         1.5|
+------+----------+------+----------+------------+



In [ ]:
#@title Q4 Merge table A table B if A.

In [31]:
#@title Q5  For table C create a new column and add saving and current accnt
Q5_df = withoutscpace_tableC_df.withColumn('total', isnull(col('saving_acc')) + isnull(col('current_acct')))
Q5_df.show()

AnalysisException: ignored

In [32]:
#@title Q6 For table C create a new Column which have open_date in format YYYMMM
Q6_df = withoutscpace_tableC_df.withColumn("new date", date_format(col('open_date'), 'yyyyMM'))
Q6_df.show()

+-------+----------+------------+----------+--------+
|acct_no|saving_acc|current_acct| open_date|new date|
+-------+----------+------------+----------+--------+
|     12|     20000|       50000|2023-07-01|  202307|
|     21|     30000|       60000|2022-06-23|  202206|
|     31|     30000|        null|      null|    null|
|     13|    100000|        null|2023-07-02|  202307|
|     41|      null|        null|2023-01-10|  202301|
|     14|    320000|      600000|2022-01-05|  202201|
+-------+----------+------------+----------+--------+



In [33]:
#@title Q7 For table A group by deparment order of all column amonut > 1L
Q_7 = table_A_df.filter(col('amount') > 100000).groupBy(col('department')).agg(count(col('cif_id')).alias('count')).orderBy('count')
Q_7.show()

+----------+-----+
|department|count|
+----------+-----+
|       C2E|    1|
+----------+-----+

